# This is to demo GenAI prompt in FSI

## Financial Analysis Generation by Template

Sell side analyst needs to write analysis report regularly, and there are some template that they use. following is a sample report on Apple Q4 2020 about its quarterly financial report. 

```
Apple Q1 2021 Results Exceed Expectations; Raising FVE to $145

Financial Summary:
Apple reported stellar fiscal Q1 2021 results with revenue of $111.4 billion, up 21% year-over-year, easily beating consensus estimates. All product categories saw double-digit growth led by iPhone (up 17% to $65.6B), Services (up 24% to $15.8B), and Wearables/Home/Accessories (up 30% to $13.0B).

Recommendation:
We reiterate our Buy rating on Apple shares. The stock is currently trading at $137, providing an upside of over 15% from current levels based on our revised $145 fair value estimate.

Risk Factors:
- Potential supply chain disruptions if COVID-19 situation worsens
- Intense competition, especially in 5G smartphones
- Regulatory scrutiny around App Store policies

In [1]:
from IPython.display import HTML

pdf_path = 'reports/Apple-FY21-Q1-Financial-Statement.pdf'
html = f'<iframe src="{pdf_path}" width="100%" height="600px"></iframe>'
HTML(html)


/opt/conda/lib/python3.10/site-packages/IPython/core/display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


GenAI might be able to help with prompt below.

In [2]:
pip install pdfplumber

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pdfplumber

# Open the PDF file
with pdfplumber.open('reports/Apple-FY21-Q1-Financial-Statement.pdf') as pdf:
    # Initialize an empty string to store the text
    financial_report_text = ''

    # Loop through each page and extract the text
    for page in pdf.pages:
        financial_report_text += page.extract_text()

# Print the extracted text
print(financial_report_text)

Apple Inc.
CONDENSED CONSOLIDATED STATEMENTS OF OPERATIONS (Unaudited)
(In millions, except number of shares which are reflected in thousands and per share amounts)
Three Months Ended
December 26, December 28,
2020 2019
Net sales:
Products $ 95,678 $ 79,104
Services 15,761 12,715
Total net sales (1) 111,439 91,819
Cost of sales:
Products 62,130 52,075
Services 4,981 4,527
Total cost of sales 67,111 56,602
Gross margin 44,328 35,217
Operating expenses:
Research and development 5,163 4,451
Selling, general and administrative 5,631 5,197
Total operating expenses 10,794 9,648
Operating income 33,534 25,569
Other income/(expense), net 45 349
Income before provision for income taxes 33,579 25,918
Provision for income taxes 4,824 3,682
Net income $ 28,755 $ 22,236
Earnings per share:
Basic $ 1.70 $ 1.26
Diluted $ 1.68 $ 1.25
Shares used in computing earnings per share:
Basic 16,935,119 17,660,160
Diluted 17,113,688 17,818,417
(1) Net sales by reportable segment:
Americas $ 46,310 $ 41,367
Eur

In [5]:
prompt = f"""
You are a financial analyst tasked with analyzing a company's financial report and generating an analyst report. The financial report is provided in PDF format.

Instructions:
1. Carefully read and analyze the PDF financial report, paying close attention to key financial statements, ratios, and other relevant information.

<financial_report>
{financial_report_text}
</financial_report>

2. Generate an analyst report with the following sections:

Financial Report Summary:
- Provide a concise summary of the company's financial performance, highlighting key metrics, trends, and notable figures from the financial statements.
- Discuss the company's revenue, profitability, cash flow, and balance sheet strength.

Buy/Sell Recommendation:
- Based on your analysis, provide a clear recommendation on whether an investor should buy, sell, or hold the company's stock.
- You estimated fair value (FVE), compare it with current apple share price is $130.
- Support your recommendation with specific evidence and reasoning from the financial report analysis.

Risk Alert/Warning:
- Identify and discuss any significant risks, concerns, or potential red flags that emerged from your analysis of the financial report.
- Explain the potential impact of these risks on the company's future performance and valuation.

3. Ensure that your analyst report is well-structured, easy to understand, and provides actionable insights for investors and stakeholders.

4. Double-check your analysis and recommendations for accuracy and consistency with the information provided in the financial report.

Please let me know if you need any clarification or have additional requirements for the analyst report.

"""

In [6]:
import boto3
import json

session = boto3.Session(region_name='us-west-2') 
bedrock = session.client(service_name="bedrock-runtime")

messages = [{"role": "user", "content": prompt}]
body = {"messages": messages, 
        "max_tokens": 10000, 
        "temperature": 0.5, 
        "anthropic_version":"", 
        "top_k": 250, 
        "top_p": 1, 
        "stop_sequences": ["Human"]}
response = bedrock.invoke_model(modelId="anthropic.claude-3-sonnet-20240229-v1:0", body=json.dumps(body))
text_resp = json.loads(response['body'].read().decode('utf-8'))
print(text_resp['content'][0]['text'])



Financial Report Summary:

Apple Inc. reported strong financial performance for the three months ended December 26, 2020. The company's total net sales increased by 21.4% year-over-year to $111.44 billion, driven by growth across all product categories and geographic segments. The iPhone remained the largest contributor to revenue, generating $65.60 billion in sales, up 17.3% from the previous year.

The company's profitability remained robust, with a gross margin of 39.8% and an operating income of $33.53 billion, representing year-over-year increases of 25.8% and 31.2%, respectively. Net income rose by 29.3% to $28.76 billion, resulting in diluted earnings per share of $1.68, up 34.4% compared to the prior year.

Apple's cash flow from operations was strong at $38.76 billion, reflecting the company's ability to generate substantial cash from its operations. The company's balance sheet remained solid, with total assets of $354.05 billion and total liabilities of $287.83 billion. The c